In [1]:
import pandas as pd
from collections import Counter
import numpy as np
from copy import deepcopy
import random
from functools import reduce
import pickle
from sklearn.model_selection import train_test_split
random.seed(42)

#  noisy testset generation

In [2]:
train_replace_dict = {'atis_flight#atis_airfare': 'atis_airfare', 
                                     'atis_ground_service#atis_ground_fare': 'atis_ground_fare',
                                     'atis_aircraft#atis_flight#atis_flight_no': 'atis_flight_no',
                                     'atis_airfare#atis_flight_time':'atis_flight_time',
                                     'atis_airline#atis_flight_no':'atis_flight_no',
                                     'atis_flight_no#atis_airline' : 'atis_flight_no',
                                     'atis_airfare#atis_flight': 'atis_airfare',
                                     'atis_flight#atis_airline' : 'atis_airline'}

# intent to idx mapping
intent_list = (pd.read_csv('../../data/ATIS/intent_list.tsv',sep='\t',header=None,names=['INTENT']).INTENT.values.tolist())
intent2idx_ATIS = { intent:idx for idx,intent in enumerate(intent_list)}

# slot idx mapping
final_slots = pd.read_csv('../../data/ATIS/slots_list.tsv',sep='\t',header=None,names=['SLOTS']).SLOTS.values.tolist()
slots2idx_ATIS  = {slots:idx for idx,slots in enumerate(final_slots)}
idx2slots = {v: k for k, v in slots2idx_ATIS.items()}

In [3]:
def mapping2idx(df,intent2idx, slots2idx):

    # adding intent encoding
    intentID = []
    for intent in list(df.INTENT):
        try:
            intentID.append(intent2idx[intent])
        except:
            print(intent)
            intentID.append(intent2idx['UNK'])
            
        
    df['INTENT_ID'] = intentID 

    # adding mapped slot column
    slots_ID = []

    for annotations in list(df.SLOTS):
        slotID = ""

        for tokens in annotations.split():
            try: 
                slotID += str(slots2idx[tokens]) + " "
            except:
                print('token',tokens)
                slotID += str(slots2idx['O']) + " "
                
        slots_ID.append(" ".join(slotID.split()))

    df['SLOTS_ID'] = slots_ID
    
    return df

In [4]:
def get_carrier_phrase_length(annotations):
    CP_idx = []
    for idx,token in enumerate(annotations.split()):
        if token == 'O':
            CP_idx.append(idx)
    
    return CP_idx 

In [5]:
def carrier_aug(data,tau):
    
    orig_data = deepcopy(data)
    
    augINTENT, augSLOTS, augTEXT,augID = [],[],[],[]
    
    cnt = 0
    for sample in data.values.tolist():
        
        
        CP_idx = get_carrier_phrase_length(sample[3])
        
        CP_length = len(CP_idx)
        
        if CP_length <= 2:
            continue
        else:
            del_count = int(CP_length/2) if CP_length <= 5 else int(tau*CP_length)
        
            del_index = random.sample(CP_idx,del_count)

            TEXT = ' '.join([i for j, i in enumerate(sample[1].split()) if j not in del_index])
            SLOTS = ' '.join([i for j, i in enumerate(sample[3].split()) if j not in del_index])
         
            augINTENT.append(sample[2])
            augTEXT.append(TEXT)
            augSLOTS.append(SLOTS)
            augID.append(cnt)
            cnt+=1
    
               
    augPD = pd.DataFrame([augID,augTEXT,augINTENT,augSLOTS],index=['ID','TEXT','INTENT','SLOTS']).T
    
    return augPD

In [6]:
def OOC_aug(data_path,out_path,fn,tau):
    
    # loading dataset
    data = pd.read_csv(data_path,sep='\t',header=0)
    
    # lowerCasing the TEXT column
    data['TEXT'] = data['TEXT'].str.lower()
    
    # handling multi-label instances 
    data['INTENT'] = data['INTENT'].replace(train_replace_dict)
    
    aug_data = carrier_aug(data,tau=tau)
    
    aug_data = mapping2idx(aug_data, intent2idx_ATIS, slots2idx_ATIS)
    
    aug_data = aug_data.sample(frac=1).reset_index(drop=True)
    aug_data.to_csv(out_path+fn,sep='\t',index=False)

In [8]:
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/20per/','test_01.tsv',tau=0.20)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/20per/','test_02.tsv',tau=0.20)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/20per/','test_03.tsv',tau=0.20)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/20per/','test_04.tsv',tau=0.20)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/20per/','test_05.tsv',tau=0.20)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/40per/','test_01.tsv',tau=0.40)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/40per/','test_02.tsv',tau=0.40)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/40per/','test_03.tsv',tau=0.40)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/40per/','test_04.tsv',tau=0.40)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/40per/','test_05.tsv',tau=0.40)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/60per/','test_01.tsv',tau=0.60)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/60per/','test_02.tsv',tau=0.60)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/60per/','test_03.tsv',tau=0.60)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/60per/','test_04.tsv',tau=0.60)
OOC_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseMC/60per/','test_05.tsv',tau=0.60)

atis_day_name
atis_day_name
token I-flight_number
token B-compartment
token B-stoploc.airport_code
token I-state_name
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token I-flight_number
token B-compartment
token B-stoploc.airport_code
token I-state_name
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token I-flight_number
token B-compartment
token B-stoploc.airport_code
token I-state_name
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token I-flight_number
token B-compartment
token B-stoploc.airport_code
token I-state_name
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token I-flight_number
token B-compartment
token B-stoploc.airport_code
token I-state_name
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token I-flight_number
token B-compartment
token B-stoploc.airport_code
token I-state_name
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token I-flight_number
token B-compartment


# SNIPS noisy testset generation

In [8]:
# intent to idx mapping
intent_list = (pd.read_csv('../../data/SNIPS/intent_list.tsv',sep='\t',header=None,names=['INTENT']).INTENT.values.tolist())
intent2idx_SNIPS = { intent:idx for idx,intent in enumerate(intent_list)}

# slot idx mapping
final_slots = pd.read_csv('../../data/SNIPS/slots_list.tsv',sep='\t',header=None,names=['SLOTS']).SLOTS.values.tolist()
slots2idx_SNIPS  = {slots:idx for idx,slots in enumerate(final_slots)}
idx2slots = {v: k for k, v in slots2idx_SNIPS.items()}

In [9]:
def OOC_aug(data_path,out_path,fn,tau):
    
    # loading dataset
    data = pd.read_csv(data_path,sep='\t',header=0)
    
    # lowerCasing the TEXT column
    data['TEXT'] = data['TEXT'].str.lower()
    
    aug_data = carrier_aug(data,tau=tau)
    
    aug_data = mapping2idx(aug_data, intent2idx_SNIPS, slots2idx_SNIPS)
    
    aug_data = aug_data.sample(frac=1).reset_index(drop=True)
    aug_data.to_csv(out_path+fn,sep='\t',index=False)

In [10]:
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/20per/','test_01.tsv',tau=0.20)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/20per/','test_02.tsv',tau=0.20)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/20per/','test_03.tsv',tau=0.20)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/20per/','test_04.tsv',tau=0.20)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/20per/','test_05.tsv',tau=0.20)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/40per/','test_01.tsv',tau=0.40)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/40per/','test_02.tsv',tau=0.40)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/40per/','test_03.tsv',tau=0.40)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/40per/','test_04.tsv',tau=0.40)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/40per/','test_05.tsv',tau=0.40)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/60per/','test_01.tsv',tau=0.60)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/60per/','test_02.tsv',tau=0.60)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/60per/','test_03.tsv',tau=0.60)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/60per/','test_04.tsv',tau=0.60)
OOC_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseMC/60per/','test_05.tsv',tau=0.60)